#### Decision-Tree（决策树）
    优点：计算复杂度不高，输出结果易于理解，对中间值得缺失不敏感，可以处理不相关特征数据
    缺点：可能会产生过度匹配问题
    适用数据类型：数值型和标称型

In [15]:
import Ipynb_importer

In [16]:
from treePlotter import *

importing Jupyter notebook from treePlotter.ipynb


### 以上两行是为了在jupyter notebook中像普通python文件一样使用import 功能，Ipynb_importer是一个解析文件

In [17]:
from math import log

In [18]:
import operator

### 在划分数据集前后信息发生的变化称为信息增益，知道如何计算增益，我们就可以计算                                                              每个特征值划分数据集获得的信息增益，获得信息增益最高的特征就是最好的选择。

### 计算给定数据的香农熵

In [19]:
def calcShannonEnt(dataSet):
    #计算多维数组的长度，在此算出的相当于第一维的维度（一共几行）
    numEntries = len(dataSet)
    #创建字典
    labelCounts = {}
    #每次从数组取一行
    for featVec in dataSet:
        #featVec[-1]表示取出来的这一行中的倒数第一个元素作为字典的键
        currentLabel = featVec[-1]
        #判断这个键在不在字典里，labelCounts.keys()返回字典里所有的键
        if currentLabel not in labelCounts.keys():
            #若不在，则增加
            labelCounts[currentLabel] = 0
        #修改该键的键值
        labelCounts[currentLabel] += 1
    shannonEnt = 0.0
    #一次取出每个键和该键的键值，键值标书其出现的频数，并用频数计算其出现的频率
    for key in labelCounts:
        prob = float(labelCounts[key])/numEntries
        #用概率计算香农熵
        shannonEnt -= prob * log(prob,2)
    return shannonEnt

#### 计算给定数据集的香农熵

#### 创建数据集 

In [20]:
def createDataSet():
    dataSet = [[1, 1, 'yes'],
               [1, 1, 'yes'],
               [1, 0, 'no'],
               [0, 1, 'no'],
               [0, 1, 'no']]
    labels = ['no surfacing', 'flippers']
    return dataSet, labels

#### 按照给定特征划分数据集 

In [21]:
def splitDataSet(dataSet, axis, value):
    retDataSet = []
    #依次取出数据集的每一行
    for featVec in dataSet:
        #比较取出的一行的第一个元素和value是否相等
        if featVec[axis] == value:
            #featVec[:axis]中[:axis]表示的是切片索引，指取出该行数据0到aixs-1的元素
            #若aixs为0，则featVec[:aixs]表示[],该语句实际上创建了一个列表
            reducedFeatVec = featVec[:axis]
            #向列表reducedFeatVec添加元素，添加的元素为featVec中从axis+1开始到结束
            reducedFeatVec.extend(featVec[axis+1 :])
            #向列表retDataSet中添加元素
            retDataSet.append(reducedFeatVec)
    return retDataSet

#### 选择最好的数据划分方式 (实现选取特征，划分数据集，计算出最好的划分数据集的特征)

In [22]:
def chooseBestFeatureToSplit(dataSet):
    #获取数据集特征的数目，不包含最后一列的类标签（只用求一行的就可以确定）
    numFeature = len(dataSet[0]) -1
    #计算数据集的信息熵（未划分之前）
    baseEntropy = calcShannonEnt(dataSet)
    #定义最优信息增益
    bestInfoGain = 0.0
    #定义最优特征值
    bestFeature = -1
    #遍历所有的特征值
    for i in range(numFeature):
        #抽取第i个特征值得特征值列表
        featList = [example[i] for example in dataSet]
        
        #使用python原生set（集合）数据类型，该类型与列表类似，不同之处在于集合类型中的每个值互不相同
        #从列表中穿建集合是python中得到列表中唯一元素值的最快方法
        uniqueVals = set(featList)
        #定义新的信息熵
        newEntropy = 0.0
        #遍历当前特征值列表中的所有唯一属性值值
        for value in uniqueVals:
            #用第i个特征值列表中的每一个唯一属性值划分一次数据集
            subDataSet = splitDataSet(dataSet, i, value)
            #计算对应该属性划分的子集占数据集的比例
            prob = len(subDataSet)/float(len(dataSet))
            #当前子集的信息熵乘以其对应的占比，并进行累加求得第i个特征值列表信息熵
            newEntropy += prob * calcShannonEnt(subDataSet)
        #计算信息增益（数据集划分前后信息发生的变化）
        infoGain = baseEntropy - newEntropy
        #与最优信息增益进行比较，若大于，则更新最优信息增益和最优特征值（信息增益最大的特征即是最好的选择）
        if(infoGain > bestInfoGain):
            bestInfoGain = infoGain
            bestFeature = i
    #返回最优特征值
    return bestFeature

#### 与chp2的classify0部分类似，该函数使用分类名称的列表，                                                                                                                               然后创建键为classList中唯一值的数据字典，字典对象中存储了                                                                                                                                  classList中每个类标签出现的频率（也即频数），最后利用                                                                                                                                            operator操作键值排序字典，并返回出现次数最多的分类名称

In [23]:
def  majorityCnt(classList):
    classCount = {}
    for vote in classList:
        if vote not in classCount.keys():
            classCount[vote] = 0
        classCount[vote] += 1
    sortedClassCount = sorted(classCount.iteritems(), key=operator.itemgetter(1), reverse=True)
    return sortedClassCount[0][0]

####  创建树

In [24]:
#labels标签列表，包含了数据集中所有特征的标签
def createTree(dataSet, labels):
    #包含数据集的所有类标签
    classList = [example[-1] for example in dataSet]
    #统计在classList列表中，类标签为classList[0]有几个，若个数于classList的个数相等，则说明所有的类标签完全相同
    #作为递归函数的第一个停止条件，直接返回该类标签
    if classList.count(classList[0]) == len(classList):
        return classList[0]
    #递归函数的第二个停止条件是使用完了所有特征，仍然不能将数据集划分成仅包含唯一类别的分组
    #假设原先dataSet[0] = [1, 1, 'yes']，此时len（dataSet[0]） = 3,其中有两个特征（分别是第一列和第二列），
    #最后一列是分类标签，若两个特征都用完了，则只剩最后一列的分类标签，所以len（dataSet[0]） == 1可以判断
    #是否用完了所有特征
    if len(dataSet[0]) == 1:
        #由于第二个条件无法简单地返回唯一的类标签，这里使用前面的majorityEnt函数挑选出现次数最多的类别作为返回值
        return majorityCnt(classList)
    #当前数据选取的最好特征值的下标
    bestFeat = chooseBestFeatureToSplit(dataSet)
    #存储最好特征值下标的对应元素
    bestFeatLabel = labels[bestFeat]
    #存储了树的所有信息
    myTree = {bestFeatLabel:{}}
    #删除对应列表下标的元素
    del(labels[bestFeat])
    #遍历当前选择特征包含的所有属性值（例如当前特征为no surfacing，其属性值有yes和no）
    featValues = [example[bestFeat] for example in dataSet]
    #从列表中创建集合使得列表中的元素值都是唯一值
    uniqueVals = set(featValues)
    #例如根据当前特征值no surfacing 来选择，可以获得5个属性值（其中三个为yes，两个为no），yes为1，no为0,即[1, 1, 1, 0, 0]
    #利用set函数将重复的值去掉，所以uniqueVals =  [1,0]
    #遍历uniqueVals
    for value in uniqueVals:
        #由于在前面labels中已删去了当前用于划分的最好特征值no surfacing，因此此时labels[:]中就只剩下Flippers
        subLabels = labels[:]
        #根据当前的最好特征值no surfacing划分的数据集分为yes（3个）和no（2个），分别对归为yes和no的数据集再次递归调用createTree()函数
        #直到其不能再划分，得到的返回值将被插入到字典myTree
        myTree[bestFeatLabel][value] = createTree(splitDataSet(dataSet,bestFeat, value), subLabels)
    return myTree

####  决策树分类函数 


#### 在执行数据分类时，需要决策树以及用于构造树的标签向量。然后，程序比较测试数据（testVec参数表示划分数据集的第一和第二特征的属性值）与决策树上的数值，递归执行该过程直到进人叶子节点；最后将测试数据定义为叶子节点所属的类型。

In [25]:
def classify(inputTree,featLabels,testVec):  
    # 得到树中的第一个特征  
    firstStr = inputTree.keys()[0]  
    # 得到第一个对应的值  
    secondDict = inputTree[firstStr]  
    # 得到树中第一个特征对应的索引  
    featIndex = featLabels.index(firstStr)  
    # 遍历secondDict字典的键
    for key in secondDict.keys():  
        # 如果在secondDict[key]中找到testVec[featIndex] ，testVec参数表示划分数据集的第一和第二特征的属性值 
        if testVec[featIndex] == key:  
            # 判断secondDict[key]是否为字典  
            if type(secondDict[key]).__name__ == 'dict':  
                # 若为字典，递归继续 
                classLabel = classify(secondDict[key], featLabels, testVec)  
            else:  
                # 若secondDict[key]为标签值即叶子节点，则将secondDict[key]赋给classLabel  
                classLabel = secondDict[key]  
    # 返回类标签  
    return classLabel  

#### 使用pick模块存储决策树 

In [33]:
# 决策树的序列化  
def storeTree(inputTree,filename):  
    # 导入pyton模块  
    import pickle  
    # 以写的方式打开文件  
    fw = open(filename,'w')  
    # 决策树序列化  
    pickle.dump(inputTree,fw)   

In [34]:
# 读取序列化的树          
def grabTree(filename):  
    import pickle  
    fr = open(filename)  
    # 返回读到的树  
    return pickle.load(fr)  

#### 使用决策树预测隐形眼镜类型

In [39]:
#收集数据，打开存放数据集的文件
fr = open('lenses.txt')
#准备数据集，读取文件，一次读一行存入inst ，strip()移除字符串头尾指定的字符（默认为空格）
#split()通过指定分隔符对字符串进行切片，如果参数num 有指定值，则仅分隔 num 个子字符串
#返回分割后的字符串列表
lenses = [inst.strip().split('\t') for inst in fr.readlines()]
#标签列表包含了数据集中所有特征的标签
lensesLabels = ['age','prescipt','astigmatic','tearRate']
#创建树
lensesTree = createTree(lenses, lensesLabels)
#绘制树
createPlot(lensesTree)